# Tá rodando

## Instalando bibliotecas necessárias

https://github.com/mastqe/tsplib/blob/master/kroA100.tsp

Essa aqui instala a biblioteca que puxa os dados das cidades do caixeiro viajante desse repositório no Github.

In [ ]:
!pip install tsplib95

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed

Esse código verifica se a biblioteca foi instalada com sucesso.

In [ ]:
!pip show tsplib95

Name: tsplib95
Version: 0.7.1
Summary: TSPLIB95 works with TSPLIB95 files.
Home-page: https://github.com/rhgrant10/tsplib95
Author: Robert Grant
Author-email: rhgrant10@gmail.com
License: Apache Software License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: Click, Deprecated, networkx, tabulate
Required-by: 


Esse outro instala biblioteca de requisição. Para baixar o arquivo de dados do problema TSP

In [ ]:
!pip install requests

Esse código verifica se a biblioteca `requests` foi instalada com sucesso.

In [ ]:
!pip show requests

Name: requests
Version: 2.31.0
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: bigframes, CacheControl, community, earthengine-api, fastai, folium, gcsfs, gdown, geocoder, google-api-core, google-cloud-bigquery, google-cloud-storage, google-colab, huggingface-hub, kaggle, kagglehub, moviepy, music21, pandas-datareader, panel, pooch, pymystem3, requests-oauthlib, spacy, Sphinx, tensorboard, tensorflow-datasets, torchtext, transformers, tweepy, weasel, yfinance


Esse outro instala biblioteca para representar o problema TSP como um grafo. Foi sugestão do Gemini

In [ ]:
!pip install networkx

## Código funcional kroA100
- 50 gerações + 10 SOS iterações
- se vcs precisarem mudar, fica na linha **256**.
- parâmetros do SOS_ACO
  * `num_ants`=10,
  * `num_iterations`=50,
  * `decay`=0.95,
  * `alpha`=1.0,
  * `beta`=2.0,
  * `population_size`=20,
  * `sos_iterations`=10

In [ ]:
import numpy as np # Para trabalhar com matrizes e operações numéricas
import random # Geração de números aleatórios (usado na inicialização e seleção)
import time # Para medir o tempo de execução do algoritmo.
import matplotlib.pyplot as plt # Para visualização (gráfico de barras)

#depende do !pip antes
import tsplib95 #  Leitura e manipulação de problemas do TSP em formato TSPLIB.
import requests # Para baixar o arquivo de dados do problema TSP.
import networkx as nx # Para representar o problema TSP como um grafo.

"""
Classe do Algoritmo Ant Colony Optimization (ACO)

Parâmetros:
--> num_ants - número de formigas
--> num_iterations - número de iterações (ou gerações)
--> decay - Taxa de evaporação do feromônio (ρ).
--> alpha -  Importância do feromônio na escolha do caminho (α)
--> beta - Importância da distância na escolha do caminho.

"""
class ACO:
    def __init__(self, num_ants, num_iterations, decay, alpha, beta): #decay = ro
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    # Método principal que executa o algoritmo ACO. Recebe a matriz de distâncias entre as cidades.
    # Inicializa os feromônios em todas as arestas.
    # Em cada iteração:
      # 1) Constrói soluções (caminhos) para cada formiga.
      # 2) Atualiza os feromônios com base nas soluções encontradas.
      # 3) Encontra o melhor caminho encontrado até o momento.
      # 4) Retorna o melhor caminho e sua distância total.

    def run(self, distances):
        num_cities = len(distances)
        pheromones = np.ones((num_cities, num_cities)) / num_cities
        best_path = None
        best_distance = float('inf')

        for iteration in range(self.num_iterations):
            all_paths = self._construct_solutions(pheromones, distances)
            self._update_pheromones(pheromones, all_paths, distances)
            best_path, best_distance = self._find_best_path(all_paths, distances, best_path, best_distance)
            print(f"Iteration {iteration+1}/{self.num_iterations}, Best Distance: {best_distance}")

        return best_path, best_distance

    def _construct_solutions(self, pheromones, distances):
        all_paths = [self._generate_path(pheromones, distances) for _ in range(self.num_ants)]
        return all_paths

    def _generate_path(self, pheromones, distances): # função que gera o caminho com feromônio
        num_cities = len(distances)
        path = []
        visited = set()
        current_city = random.randint(0, num_cities - 1)
        path.append(current_city)
        visited.add(current_city)

        while len(visited) < num_cities:
            next_city = self._select_next_city(current_city, pheromones, distances, visited)
            path.append(next_city)
            visited.add(next_city)
            current_city = next_city

        return path

    def _select_next_city(self, current_city, pheromones, distances, visited):
        num_cities = len(pheromones)
        probabilities = np.zeros(num_cities)

        valid_cities = []  # Armazenar cidades válidas (aquelas não visitadas e com distância finita)

        for city in range(num_cities):
            if city not in visited and distances[current_city][city] != float('inf'):
                distance = distances[current_city][city]
                probabilities[city] = (pheromones[current_city][city] ** self.alpha) * \
                                      ((1.0 / distance) ** self.beta)
                valid_cities.append(city)

        if not valid_cities:  # Tratar o caso em que nenhuma cidade válida foi encontrada
            return random.choice(list(set(range(num_cities)) - visited))  # Escolha aleatoriamente entre as cidades restantes

        probabilities = probabilities / probabilities.sum()  # Normaliza probabilidades apenas para cidades válidas
        return np.random.choice(valid_cities, p=probabilities[valid_cities])  # Escolhe das cidades válidas

    def _update_pheromones(self, pheromones, all_paths, distances): # atualiza os feromonios
        num_cities = len(distances)
        pheromones *= self.decay  # no artigo é o ro

        for path in all_paths:
            path_length = self._calculate_path_length(path, distances)
            for i in range(len(path) - 1):
                pheromones[path[i]][path[i + 1]] += 1.0 / path_length
                pheromones[path[i + 1]][path[i]] += 1.0 / path_length

    def _calculate_path_length(self, path, distances): # calcula o comprimento do caminho
        length = sum(distances[path[i]][path[i + 1]] for i in range(len(path) - 1))
        length += distances[path[-1]][path[0]]
        return length

    def _find_best_path(self, all_paths, distances, best_path, best_distance):
        for path in all_paths:
            length = self._calculate_path_length(path, distances)
            if length < best_distance:
                best_distance = length
                best_path = path
        return best_path, best_distance

"""
Classe do Algortimo SOS (Simbiose de Organismos):

Parâmetros:
--> population_size - tamanho da população
--> iterations - número de iterações (ou gerações)

"""
class SOS:
    def __init__(self, population_size, iterations): #iniciializa
        self.population_size = population_size
        self.iterations = iterations

    # Método principal que executa o algoritmo SOS.
      # objective_function: Função que avalia a qualidade de uma solução (no caso, a distância total do caminho).
      # Inicializa a população com soluções aleatórias.
      # Em cada iteração, para cada organismo:
          # 1) Aplica os relacionamentos de mutualismo, comensalismo e parasitismo.
          # 2) Atualiza o organismo com base nos relacionamentos.
          # 3) Atualiza a melhor solução encontrada.
          # 4) Retorna a melhor solução.

    def optimize(self, objective_function): # define a função objetivo
        population = [self._initialize_solution() for _ in range(self.population_size)]
        best_solution = None
        best_fitness = float('inf')

        for iteration in range(self.iterations):
            for i in range(self.population_size):
                organism = population[i]
                fitness = objective_function(organism) # calcula o fitness

                if fitness < best_fitness:
                    best_fitness = fitness
                    best_solution = organism

                j = random.randint(0, self.population_size - 1)
                if i != j:
                    organism, population[j] = self._mutualism(organism, population[j], objective_function)

                j = random.randint(0, self.population_size - 1)
                if i != j:
                    organism = self._commensalism(organism, population[j], objective_function)

                j = random.randint(0, self.population_size - 1)
                if i != j:
                    organism = self._parasitism(organism, population[j], objective_function)

                population[i] = organism
            print(f"Iteration {iteration+1}/{self.iterations}, Best Fitness: {best_fitness}")

        return best_solution

    def _initialize_solution(self):
        alpha = random.uniform(0.5, 2.0)
        beta = random.uniform(0.5, 2.0)
        return (alpha, beta)

    # Realiza o mutualismo
    def _mutualism(self, organism1, organism2, objective_function):
        new_organism1 = self._adjust_solution(organism1)
        new_organism2 = self._adjust_solution(organism2)
        return new_organism1, new_organism2

    # Realiza o Comensalismo
    def _commensalism(self, organism, other_organism, objective_function):
        new_organism = self._adjust_solution(organism)
        return new_organism

    # Realiza o Parasitismo
    def _parasitism(self, organism, other_organism, objective_function):
        new_organism = self._adjust_solution(organism)
        return new_organism

    # Faz um ajuste na solução para os mínimos e máximos de aplha e beta
    def _adjust_solution(self, solution):
        alpha, beta = solution
        alpha += random.uniform(-0.1, 0.1)
        beta += random.uniform(-0.1, 0.1)
        alpha = max(0.1, min(2.0, alpha))
        beta = max(0.1, min(2.0, beta))
        return (alpha, beta)

"""
Classe do Algortimo híbrido SOS-ACO (Simbiose de Organismos com Ant Colony Optimization):

Parâmetros:
--> num_ants - número de formigas
--> num_iterations - número de iterações (ou gerações)
--> decay - fator de decaimento ou o Rô
--> alpha, beta - parâmetros de pheromone
--> population_size - tamanho da população
--> sos_iterations - número de iterações (ou gerações) do algoritmo SOS

"""

class SOS_ACO:
    def __init__(self, num_ants, num_iterations, decay, alpha, beta, population_size, sos_iterations):
        self.aco = ACO(num_ants, num_iterations, decay, alpha, beta)
        self.sos = SOS(population_size, sos_iterations)

    # Combina ACO e SOS para otimizar os parâmetros do ACO (alpha e beta).
    # Utiliza o SOS para encontrar os melhores valores de alpha e beta que minimizam a distância total do caminho encontrado pelo ACO.
    # Execução do Experimento:
      # 1) Baixa o arquivo de dados do problema TSP "kroA100.tsp".
      # 2) Carrega o problema usando a biblioteca tsplib95.
      # 3) Executa o algoritmo híbrido SOS-ACO.
      # 4) Imprime e (opcionalmente) visualiza os resultados.
    def run(self, distances):
        def objective_function(params):
            alpha, beta = params
            self.aco.alpha = alpha
            self.aco.beta = beta
            _, distance = self.aco.run(distances)
            return distance

        best_params = self.sos.optimize(objective_function)
        self.aco.alpha, self.aco.beta = best_params
        best_path, best_distance = self.aco.run(distances)
        return best_path, best_distance

# Começa aqui a impressão dos resultados da execução
def print_results(experimental, article):
    print("\nExperimental Results:")
    for name, data in experimental.items():
        print(f"- {name}: Distance = {data['distance']:.2f} ± {data['std_dev']:.2f}, Time = {data['time']:.2f} seconds")

    print("\nArticle Results:")
    for name, data in article.items():
        print(f"- {name}: Distance = {data['distance']}, Time = {data.get('time', 'N/A')}")

# Começa aqui a execução do experimento
def run_experiment(problem, num_runs=10):
    best_distances = []
    total_time = 0  # Acompanhe o tempo total de todas as execuções

    # Obtenha distâncias como um array NumPy para indexação eficiente
    graph = problem.get_graph()
    distances = nx.to_numpy_array(graph)  # Converter gráfico NetworkX em matriz NumPy

    for _ in range(num_runs):
        start_time = time.time()
        # é aqui que altera os parâmetros gerais de execução
        sos_aco = SOS_ACO(num_ants=10, num_iterations=50, decay=0.95, alpha=1.0, beta=2.0, population_size=20, sos_iterations=10)
        best_path, best_distance = sos_aco.run(distances)  # Use a matriz NumPy para distâncias
        best_distances.append(best_distance)

        end_time = time.time()
        total_time += (end_time - start_time)

    # calculo da média da distancia
    avg_distance = np.mean(best_distances)
    std_dev = np.std(best_distances)
    avg_time = total_time / num_runs
    return avg_distance, std_dev, avg_time


# Baixe e processe o conjunto de dados
url = 'https://raw.githubusercontent.com/mastqe/tsplib/master/kroA100.tsp'
response = requests.get(url)

with open('kroA100.tsp', 'w') as f: # cria 1 arquivo temporário do kroA100
    f.write(response.text)

problem = tsplib95.load('kroA100.tsp')

# Execute o experimento e armazene os resultados
mean_kroa100, std_kroa100, time_kroa100 = run_experiment(problem)

experimental_results = {
    "kroA100": {"distance": mean_kroa100, "std_dev": std_kroa100, "time": time_kroa100}  # Adicionado desvio padrão
}

# Dados de referência do artigo (substituir por valores reais)
article_results = {
    "kroA100": {"distance": 21282}  # Substitua pela distância real do artigo
}

# Saída e visualização
print_results(experimental_results, article_results)

# Visualizar resultados (opcional) #ignorem esse gráfico q tá ruim. Não deu tempo de melhorar
plt.bar(experimental_results.keys(), [v['distance'] for v in experimental_results.values()])
plt.ylabel('Tour Distance')
plt.title('Experimental Results (SOS-ACO)')
plt.show()


Iteration 1/50, Best Distance: 52290.0
Iteration 2/50, Best Distance: 51473.0
Iteration 3/50, Best Distance: 51473.0
Iteration 4/50, Best Distance: 51473.0
Iteration 5/50, Best Distance: 51473.0
Iteration 6/50, Best Distance: 51473.0
Iteration 7/50, Best Distance: 51473.0
Iteration 8/50, Best Distance: 51473.0
Iteration 9/50, Best Distance: 51473.0
Iteration 10/50, Best Distance: 51473.0
Iteration 11/50, Best Distance: 47747.0
Iteration 12/50, Best Distance: 47747.0
Iteration 13/50, Best Distance: 47747.0
Iteration 14/50, Best Distance: 47747.0
Iteration 15/50, Best Distance: 47747.0
Iteration 16/50, Best Distance: 47747.0
Iteration 17/50, Best Distance: 47747.0
Iteration 18/50, Best Distance: 47747.0
Iteration 19/50, Best Distance: 47747.0
Iteration 20/50, Best Distance: 47747.0
Iteration 21/50, Best Distance: 47747.0
Iteration 22/50, Best Distance: 47747.0
Iteration 23/50, Best Distance: 47747.0
Iteration 24/50, Best Distance: 47747.0
Iteration 25/50, Best Distance: 47747.0
Iteration

KeyboardInterrupt: 